**Ultralytics download**
to use TOLOv11 Model

In [1]:
import os
import random
import shutil

# Spliting dataset
def split_dataset(dataset_dir, train_ratio=0.8, val_ratio=0.2):
    images_dir = os.path.join(dataset_dir, "images")
    labels_dir = os.path.join(dataset_dir, "labels")
    
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg'))]
    random.shuffle(image_files)
    
    num_images = len(image_files)
    num_train = int(num_images * train_ratio)
    
    train_images = image_files[:num_train]
    val_images = image_files[num_train:]
    
    # Create train and val directories
    for split in ['train', 'val']:
        os.makedirs(os.path.join(dataset_dir, split, "/images"), exist_ok=True)
        os.makedirs(os.path.join(dataset_dir, split, "/labels"), exist_ok=True)

    def move_files(image_list, dest_images, dest_labels):
        for image_file in image_list:
            label_file = image_file[:-4] + ".txt"
            
            source_image = os.path.join(images_dir, image_file)
            source_label = os.path.join(labels_dir, label_file)
            
            dest_image_path = os.path.join(dest_images, image_file)
            dest_label_path = os.path.join(dest_labels, label_file)
            
            shutil.move(source_image, dest_image_path)
            if os.path.exists(source_label):
                shutil.move(source_label, dest_label_path)

    move_files(train_images, os.path.join(dataset_dir, "train", "images"), os.path.join(dataset_dir, "train", "labels"))
    move_files(val_images, os.path.join(dataset_dir, "val", "images"), os.path.join(dataset_dir, "val", "labels"))

In [2]:
import torch
from torchvision import transforms
import cv2
import numpy as np

def augment_image(image_path, output_dir, label_path, num_augments=3):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)  # Convert to PIL before transforms

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.5, contrast=0.5),
        transforms.RandomRotation(degrees=30),
        transforms.ToTensor()
    ])
    
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    for i in range(num_augments):
        aug_image = transform(image)
        aug_image = transforms.ToPILImage()(aug_image)  # Convert tensor back to PIL
        aug_filename = f"{base_name}_aug{i}.jpg"
        aug_image.save(os.path.join(output_dir, aug_filename))  # Save using PIL

        if os.path.exists(label_path):
            aug_label_filename = f"{base_name}_aug{i}.txt"
            shutil.copy(label_path, os.path.join(output_dir.replace("images", "labels"), aug_label_filename))


In [3]:
# Dataset augmentation 
def augment_dataset(dataset_dir, split="train"):
    images_dir = os.path.join(dataset_dir, split, "images")
    labels_dir = os.path.join(dataset_dir, split, "labels")
    
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg'))]
    
    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        label_path = os.path.join(labels_dir, image_file[:-4] + ".txt")
        augment_image(image_path, images_dir, label_path, num_augments=3)


In [8]:
# Example usage:
dataset_path = "./dataset" # Replace with the path to your dataset
split_dataset(dataset_path)

In [14]:
# prompt: custom dataset yaml format for yolo avoid function

import os

# Define the path to your custom dataset directory
dataset_path = "./datasets/"  # Replace with the actual path

# Create the data.yaml file
data_yaml_content = f"""
path: {dataset_path}  # dataset root dir
train: train/images  # train images (relative to 'path') 128 images
val: val/images  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
names:
  0: cardboard
  1: glass
  2: metal
  3: paper
  4: plastic

"""

# Create the data.yaml file in the dataset directory
with open(os.path.join(dataset_path, "data.yaml"), "w") as f:
  f.write(data_yaml_content)

print(f"data.yaml file created at: {os.path.join(dataset_path, 'data.yaml')}")


data.yaml file created at: ./datasets/data.yaml


In [12]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Display model information (optional)
model.info()


YOLO11n summary: 319 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(319, 2624080, 0, 6.614336)

In [ ]:
# Train the model
results = model.train(data="data.yaml", epochs=60, imgsz=640)

Ultralytics 8.3.75  Python-3.12.9 torch-2.6.0+cpu CPU (Intel Core(TM) i7-6500U 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=60, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

train: Scanning C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\datasets\train\labels... 198 images, 1 backgrounds, 0 corrupt: 100%|██████████| 198/198 [00:02<00:00, 73.22it/s]


train: New cache created: C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\datasets\train\labels.cache


val: Scanning C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\datasets\val\labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 75.64it/s]

val: New cache created: C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\datasets\val\labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/60         0G     0.8026      1.435       1.32         15        640: 100%|██████████| 13/13 [03:11<00:00, 14.75s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:12<00:00,  6.49s/it]

                   all         50         50      0.641      0.452      0.419       0.28



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      16/60         0G     0.8225      1.527      1.305         46        640:  38%|███▊      | 5/13 [25:53<1:10:34, 529.37s/it]

Save model

In [ ]:
import torch

# Save the model
torch.save(model.state_dict(), "yolo_model.pth")

Load Model

In [ ]:
# # Load the saved weights
# model.load_state_dict(torch.load("yolo_model.pth"))
# model.eval()  # Set to evaluation mode

In [ ]:
import cv2
# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam

while cap.isOpened():
    # Read a frame from the webcam
    success, frame = cap.read()

    if success:
        # Run inference on the frame
        results = model(frame)

        # Get the annotated frame with bounding boxes
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Object Detection", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        print("Error reading frame from webcam.")
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

